In [273]:
import pandas as pd 
import matplotlib as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.models import Model
from math import radians, sin, cos, sqrt, atan2

In [274]:
user = pd.read_csv('./data/user.csv')
destination = pd.read_csv('./data/destinasi-wisata-indonesia.csv')
bali =  pd.read_csv('./data/Dataset - tourismBali.csv')

In [275]:
user.head()

,User_Id,Location,Age
0,1,"Semarang, Jawa Tengah",20
1,2,"Bekasi, Jawa Barat",21
2,3,"Cirebon, Jawa Barat",23
3,4,"Bekasi, Jawa Barat",21
4,5,"Lampung, Sumatera Selatan",20


In [276]:
bali.head()

,Place_Id,Place_Name,Description,Weekend Holiday Price,Weekday Price,Category,City,Rating,Alamat,Coordinate,Lat,Long,Gambar
0,1,Agrowisata Satria,Agrowisata Satria menawarkan 'rasa' Bali pada ...,50000,50000,Agrowisata,Gianyar,3.0,"Manukaya, Kec. Tampaksiring, Kabupaten Gianya...","-8.402228546131536, 115.32296693687739",-8.402229,115.322967,V
1,2,Agung Bali,Dapatkan berbagai produk oleh-oleh khas Bali b...,0,0,Belanja,Badung,4.0,"Jln. Dewi Sri No.18XX, Kuta, Badung, Bali","-8.700234336021559, 115.176534407375",-8.700234,115.176534,V
2,3,Wisata Air Panas Toya Bungkah,Pemandian air panas di dekat gunung Batur (Pem...,70000,70000,Alam,Bangli,4.0,"Batur Tengah, Kec. Kintamani, Kabupaten Bangli...","-8.251298580809106, 115.39981444195475",-8.251299,115.399814,V
3,4,Air Terjun Aling-Aling,Keberadaan air terjun ini semakin mempercantik...,20000,20000,Alam,Buleleng,4.5,"Jl. Raya Desa Sambangan, Banjar, Sambangan, Ke...","-8.173073590748519, 115.10513024047802",-8.173074,115.105130,V
4,5,Air Terjun Tegenungan,Bosan dengan wisata alam pantai dan ingin meni...,20000,20000,Alam,Gianyar,4.0,"Jl. Ir. Sutami, Kemenuh, Kec. Sukawati, Kabupa...","-8.575191460385602, 115.28870189592831",-8.575191,115.288702,V


In [277]:
bali[['Weekday Price', 'Weekend Holiday Price']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   Weekday Price          75 non-null     int64
 1   Weekend Holiday Price  75 non-null     int64
dtypes: int64(2)
memory usage: 1.3 KB


In [278]:
mean = (bali['Weekday Price'].values + bali['Weekend Holiday Price'].values) / 2

In [279]:
mean

array([ 50000.,      0.,  70000.,  20000.,  20000.,  15000., 300000.,
       102500.,  20000., 500000.,      0.,  32500.,  50000.,      0.,
        15000.,  30000.,  10000.,  10000.,      0.,      0.,      0.,
        95000.,  30000.,  25000.,  15000.,  10000.,      0.,  40000.,
            0.,      0.,  25000.,      0.,  60000.,  25000.,  25000.,
        10000.,  10000.,      0.,      0.,  10000.,  10000.,      0.,
            0.,   5000.,      0.,   8000.,      0.,   5000.,   8000.,
            0.,      0.,  10000.,      0.,      0.,      0.,      0.,
        30000.,  30000.,  10000.,  50000.,  15000.,      0.,  15000.,
        15000.,  30000.,  50000.,      0.,      0., 100000., 150000.,
        10000.,  25000., 289000.,  10000.,      0.])

In [280]:
bali['Price'] = mean

In [281]:
bali.columns

Index(['Place_Id', 'Place_Name', 'Description', 'Weekend Holiday Price',
       'Weekday Price', 'Category', 'City', 'Rating', 'Alamat', 'Coordinate',
       'Lat', 'Long', 'Gambar', 'Price'],
      dtype='object')

In [282]:
destination.columns

Index(['Place_Id', 'Place_Name', 'Description', 'Category', 'City', 'Price',
       'Rating', 'Time_Minutes', 'Coordinate', 'Lat', 'Long', 'Column1', '_1',
       'Rating_Count'],
      dtype='object')

In [283]:
bali = bali[['Place_Id', 'Place_Name', 'Description', 'Category', 'City', 'Price', 'Rating', 'Coordinate', 'Lat', 'Long']]

In [284]:
bali['City'] = 'Bali'

In [285]:
destination = pd.concat([destination, bali], ignore_index=True)

In [286]:
destination.shape

(512, 14)

In [287]:
destination.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Column1,_1,Rating_Count
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000.0,46.0,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-61753924.0,1.068272e+09,NaN,1.0,18.0
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0.0,46.0,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-61376448.0,1.068171e+09,NaN,2.0,25.0
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000.0,46.0,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-61253124.0,1.068335e+09,NaN,3.0,19.0
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000.0,45.0,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-63024459.0,1.068952e+09,NaN,4.0,21.0
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000.0,45.0,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-612419.0,1.068391e+08,NaN,5.0,24.0


In [288]:
destination['City'].value_counts()

City
Yogyakarta    126
Bandung       124
Jakarta        84
Bali           75
Semarang       57
Surabaya       46
Name: count, dtype: int64

In [289]:
destination['Category'].unique()

array(['Budaya', 'Taman Hiburan', 'Cagar Alam', 'Bahari',
       'Pusat Perbelanjaan', 'Tempat Ibadah', 'Agrowisata', 'Belanja',
       'Alam', 'Rekreasi', 'Religius', 'Pantai'], dtype=object)

In [290]:
destination['Category'] = destination['Category'].replace('Pantai', 'Bahari')

In [291]:
destination['Category'].unique()

array(['Budaya', 'Taman Hiburan', 'Cagar Alam', 'Bahari',
       'Pusat Perbelanjaan', 'Tempat Ibadah', 'Agrowisata', 'Belanja',
       'Alam', 'Rekreasi', 'Religius'], dtype=object)

In [292]:
destination[destination['Category'] == 'Bahari' ].sample(7)

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Column1,_1,Rating_Count
9,10,Pulau Tidung,Pulau Tidung adalah salah satu kelurahan di ke...,Bahari,Jakarta,150000.0,45.0,NaN,"{'lat': -5.803205300000001, 'lng': 106.5237907}",-5.803205e+07,1.065238e+09,NaN,10.0,14.0
490,54,Pantai Seminyak,Tempat wisata unggulan di kawasan Seminyak ada...,Bahari,Bali,0.0,4.5,NaN,"-8.691753184065904, 115.15827019690434",-8.691753e+00,1.151583e+02,NaN,NaN,NaN
195,196,Pantai Ngrenehan,Pantai Ngrenehan merupakan\npantai nelayan. Te...,Bahari,Yogyakarta,3000.0,44.0,NaN,"{'lat': -8.1211475, 'lng': 110.5142337}",-8.121148e+07,1.105142e+09,NaN,196.0,21.0
176,177,Pantai Parangtritis,Pantai Parangtritis (bahasa Jawa: ꦥꦱꦶꦱꦶꦂ ꦥꦫꦁꦠꦿ...,Bahari,Yogyakarta,10000.0,45.0,NaN,"{'lat': -8.024607999999999, 'lng': 110.3298045}",-8.024608e+06,1.103298e+09,NaN,177.0,39.0
208,209,Pantai Sanglen,Pantai Sanglen. Lokasinya berada di Desa Kemad...,Bahari,Yogyakarta,10000.0,45.0,NaN,"{'lat': -8.1367456, 'lng': 110.5716362}",-8.136746e+07,1.105716e+09,NaN,209.0,14.0
478,42,Pantai Double Six,Salah satu daya tarik Pantai Double Six yang m...,Bahari,Bali,0.0,4.0,NaN,"-8.697360300916719, 115.16212766873024",-8.697360e+00,1.151621e+02,NaN,NaN,NaN
200,201,Pantai Kesirat,Pada umumnya pantai identik dengan pasirnya bu...,Bahari,Yogyakarta,5000.0,46.0,90.0,"{'lat': -8.096153500000002, 'lng': 110.4350483}",-8.096154e+07,1.104350e+09,NaN,201.0,37.0


In [293]:
destination.isnull().sum()

Place_Id          0
Place_Name        0
Description       0
Category          0
City              0
Price             0
Rating            0
Time_Minutes    307
Coordinate        0
Lat               0
Long              0
Column1         512
_1               75
Rating_Count     75
dtype: int64

In [294]:
destination.drop(columns=['Place_Id','Time_Minutes', 'Column1', '_1', 'Rating_Count'], inplace=True)

In [295]:
destination.columns

Index(['Place_Name', 'Description', 'Category', 'City', 'Price', 'Rating',
       'Coordinate', 'Lat', 'Long'],
      dtype='object')

In [296]:
destination = destination.rename(columns={'Place_Name':'place_Name',
                                          'Description':'description',
                                          'Category':'category',
                                          'City':'city',
                                          'Price':'price',
                                          'Rating':'rating_avg',
                                          'Lat': 'latitude', 
                                          'Long': 'longitude',
                                          'Coordinate':'coordinate'})

In [297]:
destination.duplicated().sum()

0

In [298]:
destination.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   place_Name   512 non-null    object 
 1   description  512 non-null    object 
 2   category     512 non-null    object 
 3   city         512 non-null    object 
 4   price        512 non-null    float64
 5   rating_avg   512 non-null    float64
 6   coordinate   512 non-null    object 
 7   latitude     512 non-null    float64
 8   longitude    512 non-null    float64
dtypes: float64(4), object(5)
memory usage: 36.1+ KB


##REKOMENDASI BY KATEGORI WISATA

### PREPROCESSING
di encode + cari kesamaan pake cosine similarity

In [299]:
oneHotCategory = OneHotEncoder(sparse=False)
oneHotCity = OneHotEncoder(sparse=False)

In [300]:
categoryEncoded = oneHotCategory.fit_transform(destination[['category']])
cityEncoded = oneHotCity.fit_transform(destination[['city']])

c:\Users\Akmal Fauzan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\Akmal Fauzan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [301]:
categoryEncoded = pd.DataFrame(categoryEncoded, columns=oneHotCategory.get_feature_names_out(['category']))
cityEncoded = pd.DataFrame(cityEncoded, columns=oneHotCity.get_feature_names_out(['city']))

In [302]:
categoryEncoded

,category_Agrowisata,category_Alam,category_Bahari,category_Belanja,category_Budaya,category_Cagar Alam,category_Pusat Perbelanjaan,category_Rekreasi,category_Religius,category_Taman Hiburan,category_Tempat Ibadah
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
507,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
510,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [303]:
cityEncoded

,city_Bali,city_Bandung,city_Jakarta,city_Semarang,city_Surabaya,city_Yogyakarta
0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
507,1.0,0.0,0.0,0.0,0.0,0.0
508,1.0,0.0,0.0,0.0,0.0,0.0
509,1.0,0.0,0.0,0.0,0.0,0.0
510,1.0,0.0,0.0,0.0,0.0,0.0


In [304]:
cosineSim = cosine_similarity(categoryEncoded)
cosineCity = cosine_similarity(cityEncoded)

In [305]:
index_combined = destination['place_Name'] + " - " + destination['city']
cosineSim = pd.DataFrame(cosineSim, index=index_combined, columns=index_combined)
cosineCity = pd.DataFrame(cosineCity, index=index_combined, columns=index_combined)

In [306]:
cosineSim

,Monumen Nasional - Jakarta,Kota Tua - Jakarta,Dunia Fantasi - Jakarta,Taman Mini Indonesia Indah (TMII) - Jakarta,Atlantis Water Adventure - Jakarta,Taman Impian Jaya Ancol - Jakarta,Kebun Binatang Ragunan - Jakarta,Ocean Ecopark - Jakarta,Pelabuhan Marina - Jakarta,Pulau Tidung - Jakarta,...,Pura Uluwatu - Bali,Pusat Kesenian Ukir Desa Mas - Bali,Watersport Tanjung Benoa Bali - Bali,Tari Barong dan Keris\r\n - Bali,Kecak and Fire Dance - Bali,Teras Sawah Tegalalang - Bali,Tirta Gangga - Bali,Waterbom Bali - Bali,Alas Harum Agro Tourism - Bali,Wisata Agro Teba Sari Bali - Bali
Monumen Nasional - Jakarta,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
Kota Tua - Jakarta,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
Dunia Fantasi - Jakarta,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taman Mini Indonesia Indah (TMII) - Jakarta,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Atlantis Water Adventure - Jakarta,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Teras Sawah Tegalalang - Bali,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
Tirta Gangga - Bali,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Waterbom Bali - Bali,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Alas Harum Agro Tourism - Bali,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


In [307]:
cosineCity

,Monumen Nasional - Jakarta,Kota Tua - Jakarta,Dunia Fantasi - Jakarta,Taman Mini Indonesia Indah (TMII) - Jakarta,Atlantis Water Adventure - Jakarta,Taman Impian Jaya Ancol - Jakarta,Kebun Binatang Ragunan - Jakarta,Ocean Ecopark - Jakarta,Pelabuhan Marina - Jakarta,Pulau Tidung - Jakarta,...,Pura Uluwatu - Bali,Pusat Kesenian Ukir Desa Mas - Bali,Watersport Tanjung Benoa Bali - Bali,Tari Barong dan Keris\r\n - Bali,Kecak and Fire Dance - Bali,Teras Sawah Tegalalang - Bali,Tirta Gangga - Bali,Waterbom Bali - Bali,Alas Harum Agro Tourism - Bali,Wisata Agro Teba Sari Bali - Bali
Monumen Nasional - Jakarta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kota Tua - Jakarta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dunia Fantasi - Jakarta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Taman Mini Indonesia Indah (TMII) - Jakarta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Atlantis Water Adventure - Jakarta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Teras Sawah Tegalalang - Bali,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Tirta Gangga - Bali,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Waterbom Bali - Bali,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Alas Harum Agro Tourism - Bali,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### Content-Based

In [308]:
def getRecommendationsByCategory(category, destination, similarity, top_n=5):
    relevant_places = destination[destination['category'] == category]

    relevant_indices = relevant_places['place_Name'] + " - " + relevant_places['city']

    if relevant_indices.empty:
        return "Tidak ada tempat wisata dengan kategori yang diminta."

    recommendations = {}
    for index in relevant_indices:
        similarityScores = similarity[index]

        sorted_scores = similarityScores.sort_values(ascending=False).iloc[1:top_n+1]
        recommendations[index] = sorted_scores

    return recommendations

In [309]:
recommendationsByCategory = getRecommendationsByCategory("Monumen", destination, cosineSim, top_n=5)
print("Rekomendasi Berdasarkan Kategori Wisata:")
print(recommendationsByCategory)

Rekomendasi Berdasarkan Kategori Wisata:
Tidak ada tempat wisata dengan kategori yang diminta.


In [310]:
def getRecommendationsByCity(city, destination, similarity, top_n=5):
    relevant_places = destination[destination['city'] == city]

    relevant_indices = relevant_places['place_Name'] + " - " + relevant_places['city']

    if relevant_indices.empty:
        return "Tidak ada tempat wisata di kota yang diminta."

    recommendations = {}
    for index in relevant_indices:
        similarityScores = similarity[index]
        
        sorted_scores = similarityScores.sort_values(ascending=False).iloc[1:top_n+1]
        recommendations[index] = sorted_scores

    return recommendations

In [311]:
recommendationsByCity = getRecommendationsByCity("Jakarta", destination, cosineCity, top_n=5)
print("\nRekomendasi Berdasarkan Kota:")
print(recommendationsByCity)


Rekomendasi Berdasarkan Kota:
{'Monumen Nasional - Jakarta': Klenteng Jin De Yuan - Jakarta           1.0
Museum Kebangkitan Nasional - Jakarta    1.0
Taman Legenda Keong Emas - Jakarta       1.0
Tugu Proklamasi - Jakarta                1.0
Museum Tekstil - Jakarta                 1.0
Name: Monumen Nasional - Jakarta, dtype: float64, 'Kota Tua - Jakarta': Klenteng Jin De Yuan - Jakarta           1.0
Museum Kebangkitan Nasional - Jakarta    1.0
Taman Legenda Keong Emas - Jakarta       1.0
Tugu Proklamasi - Jakarta                1.0
Museum Tekstil - Jakarta                 1.0
Name: Kota Tua - Jakarta, dtype: float64, 'Dunia Fantasi - Jakarta': Klenteng Jin De Yuan - Jakarta           1.0
Museum Kebangkitan Nasional - Jakarta    1.0
Taman Legenda Keong Emas - Jakarta       1.0
Tugu Proklamasi - Jakarta                1.0
Museum Tekstil - Jakarta                 1.0
Name: Dunia Fantasi - Jakarta, dtype: float64, 'Taman Mini Indonesia Indah (TMII) - Jakarta': Klenteng Jin De Yuan - Jakar

In [312]:
weightCategory = 0.5  
weightCity = 0.5

combinedSimilarity = (weightCategory * cosineSim) + (weightCity * cosineCity)

In [313]:
def getCombinedRecommendations(category, city, destination, combined_similarity, top_n=5):
    relevant_places = destination[(destination['city'] == city) & 
                                     (destination['category'] == category)]
    
    relevant_indices = relevant_places['place_Name'] + " - " + relevant_places['city']
    
    if relevant_indices.empty:
        return "Tidak ada tempat yang cocok dengan kategori dan kota yang diminta."

    recommendations = {}
    for index in relevant_indices:
        similarityScores = combined_similarity[index]
        
        sorted_scores = similarityScores.sort_values(ascending=False).iloc[1:top_n+1]
        recommendations[index] = sorted_scores
    
    return recommendations


In [314]:
budaya = destination[(destination['category'] == 'Budaya') & (destination['city'] == 'Yogyakarta')]

In [315]:
budaya

,place_Name,description,category,city,price,rating_avg,coordinate,latitude,longitude
85,Keraton Yogyakarta,Keraton Ngayogyakarta Hadiningrat atau Keraton...,Budaya,Yogyakarta,15000.0,46.0,"{'lat': -7.8052845, 'lng': 110.3642031}",-78052845.0,1.103642e+09
87,Museum Benteng Vredeburg Yogyakarta,Museum Benteng Vredeburg (bahasa Jawa: ꦩꦸꦱꦶꦪꦸꦩ...,Budaya,Yogyakarta,3000.0,46.0,"{'lat': -7.800201599999999, 'lng': 110.3663044}",-78002016.0,1.103663e+09
88,De Mata Museum Jogja,Museum De Mata merupakan salah satu museum yan...,Budaya,Yogyakarta,50000.0,44.0,"{'lat': -7.816315599999999, 'lng': 110.3871442}",-78163156.0,1.103871e+09
96,Monumen Yogya Kembali,Museum Monumen Yogya Kembali (bahasa Jawa: ꦩꦺꦴ...,Budaya,Yogyakarta,15000.0,45.0,"{'lat': -7.7495904, 'lng': 110.3696068}",-77495904.0,1.103696e+09
98,Kampung Wisata Kadipaten,Kampung Wisata Kadipaten secaara kewilayahan b...,Budaya,Yogyakarta,0.0,44.0,"{'lat': -7.806093000000001, 'lng': 110.35831}",-7806093.0,1.103583e+07
99,Taman Budaya Yogyakarta,Taman Budaya Yogyakarta (TBY) (Hanacaraka:ꦠꦩꦤ꧀...,Budaya,Yogyakarta,0.0,45.0,"{'lat': -7.8001041, 'lng': 110.3676579}",-78001041.0,1.103677e+09
100,Kampung Wisata Sosro Menduran,Kampung wisata Sosromenduran merupakan kampung...,Budaya,Yogyakarta,0.0,4.0,"{'lat': -7.792189999999999, 'lng': 110.362151}",-779219.0,1.103622e+08
101,Monumen Batik Yogyakarta,Perhatian pemerintah megenai kebudayan batik i...,Budaya,Yogyakarta,40000.0,45.0,"{'lat': -7.8011159, 'lng': 110.3646172}",-78011159.0,1.103646e+09
103,Tebing Breksi,Tebing Breksi merupakan tempat wisata yang ber...,Budaya,Yogyakarta,20000.0,44.0,"{'lat': -7.781571400000001, 'lng': 110.5045827}",-77815714.0,1.105046e+09
104,Candi Donotirto,"Walaupun melekat dengan kata candi, Candi Dono...",Budaya,Yogyakarta,0.0,44.0,"{'lat': -7.795561299999998, 'lng': 110.3612095}",-77955613.0,1.103612e+09


In [316]:
# Buat rekomendasi berdasarkan kategori wisata dan kota
recommendations_combined = getCombinedRecommendations("Budaya", "Yogyakarta", destination, combinedSimilarity, top_n=5)
print("Rekomendasi Berdasarkan Kategori dan Kota:")
print(recommendations_combined)


Rekomendasi Berdasarkan Kategori dan Kota:
{'Keraton Yogyakarta - Yogyakarta': Bangsal Pagelaran - Yogyakarta              1.0
Kampung Wisata Rejowinangun - Yogyakarta    1.0
Watu Goyang - Yogyakarta                    1.0
Candi Prambanan - Yogyakarta                1.0
Bukit Panguk Kediwung - Yogyakarta          1.0
Name: Keraton Yogyakarta - Yogyakarta, dtype: float64, 'Museum Benteng Vredeburg Yogyakarta - Yogyakarta': Bangsal Pagelaran - Yogyakarta              1.0
Kampung Wisata Rejowinangun - Yogyakarta    1.0
Watu Goyang - Yogyakarta                    1.0
Candi Prambanan - Yogyakarta                1.0
Bukit Panguk Kediwung - Yogyakarta          1.0
Name: Museum Benteng Vredeburg Yogyakarta - Yogyakarta, dtype: float64, 'De Mata Museum Jogja - Yogyakarta': Bangsal Pagelaran - Yogyakarta              1.0
Kampung Wisata Rejowinangun - Yogyakarta    1.0
Watu Goyang - Yogyakarta                    1.0
Candi Prambanan - Yogyakarta                1.0
Bukit Panguk Kediwung - Yogyakart

### Rekomendasi Terdekat

In [317]:
destination['latitude'] = destination['latitude'].apply(lambda x: float(str(x).replace("'", "").replace("{", "").replace("}", "").split(':')[-1].strip()))
destination['longitude'] = destination['longitude'].apply(lambda x: float(str(x).replace("'", "").replace("{", "").replace("}", "").split(':')[-1].strip()))

In [318]:
destination[['latitude', 'longitude']].sample(5)

,latitude,longitude
339,-7.020524e+07,1.103755e+09
11,-5.907433e+07,1.065864e+09
435,-7.294330e+07,1.127618e+09
416,-7.246944e+06,1.128022e+08
500,-8.620904e+00,1.150869e+02


In [319]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [320]:
def getTempatTerdekat(user_lat, user_long, data, top_n=5):
    data['distance_km'] = data.apply(lambda row: haversine(user_lat, user_long, row['latitude'], row['longitude']), axis=1)
    nearest_places = data.sort_values(by='distance_km').head(top_n)
    return nearest_places

In [321]:
user_lat, user_long = -8.172141e+00	, 1.149923e+02

In [322]:
nearest_places = getTempatTerdekat(user_lat, user_long, destination)

In [323]:
nearest_places[['place_Name', 'category', 'city', 'distance_km']]

,place_Name,category,city,distance_km
468,Krisna Oleh Oleh,Belanja,Bali,0.001663
440,Air Terjun Aling-Aling,Alam,Bali,12.419164
449,Bukit Wanagiri,Alam,Bali,14.781757
453,Danau Tamblingan,Alam,Bali,14.820360
445,Handara Iconic Gate,Budaya,Bali,20.350085


In [324]:
destination.to_csv('./data/destinasiWisata.csv', index=False)

### Collaborative Filtering